In [6]:

import os
import cv2
import xarray as xr
from   pathlib import Path
from   netCDF4 import Dataset
import numpy as np
import random
from PIL import Image

## 1 对比result 与 label

### 1.1 选取原始6001*6001 中切割 07_07 到23_23范围

In [3]:
path_lalo = '/home/zhoutt/public/data/H8/L1/201607/30/NC_H08_20160730_0230_R21_FLDK.06001_06001.nc' # 找一个图像读取经纬度网格
path_data = '/home/zhoutt/code/datasets/results/disk/'
img_list  = [path_data + f for f in os.listdir(path_data) if f.endswith('.npy')]
H8_lalo   = Dataset(path_lalo,mode='r')
print(H8_lalo.variables.keys())
lat_read  = H8_lalo.variables['latitude'][200*6:200*23].data
lon_read  = H8_lalo.variables['longitude'][200*6:200*23].data
SAZ_read  = H8_lalo.variables['SAZ'][200*6:200*23,200*6:200*23].data

dict_keys(['latitude', 'longitude', 'band_id', 'start_time', 'end_time', 'geometry_parameters', 'albedo_01', 'albedo_02', 'albedo_03', 'sd_albedo_03', 'albedo_04', 'albedo_05', 'albedo_06', 'tbb_07', 'tbb_08', 'tbb_09', 'tbb_10', 'tbb_11', 'tbb_12', 'tbb_13', 'tbb_14', 'tbb_15', 'tbb_16', 'SAZ', 'SAA', 'SOZ', 'SOA', 'Hour'])


### 1.2 对比全圆盘label 与test_result

In [46]:
label_path = '/home/zhoutt/code/datasets/H8label2016/H8Lb_20160730_0230_6001_6001.nc'
label_dataset  = Dataset(label_path,mode='r')
label = label_dataset.variables['cldmask'][200*6:200*23,200*6:200*23].data
label[label==6]=0
result = np.load('/home/zhoutt/code/datasets/results/disk_splice_1/20160730_0230.npy')
dif_disk_200 = result - label 
aa =  np.unique(dif_disk_200)
for i in aa:
    num = np.sum(dif_disk_200==i)
    print(i, num)
    if i==0:
        acc = num/(3400*3400)
        print('acc:',acc)

-1 200042
0 11085258
acc: 0.9589323529411765
1 274700


In [8]:
label_path = '/home/zhoutt/code/datasets/H8label2016/H8Lb_20160730_0230_6001_6001.nc'
label_dataset  = Dataset(label_path,mode='r')
label = label_dataset.variables['cldmask'][200*6:200*7,200*6:200*7].data
label[label==6]=0
result = np.load('/home/zhoutt/code/datasets/results/disk/20160730_0230.npy')[0:200,0:200]
dif_disk_200 = result - label 
aa =  np.unique(dif_disk_200)
for i in aa:
    num = np.sum(dif_disk_200==i)
    print('dif_disk_200',num)

dif_disk_200 11135
dif_disk_200 19957
dif_disk_200 8908


### 1.3 验证切割是否正确  !!!!!! 行列搞反了！！！！！😭

In [35]:
label_disk = label_dataset.variables['cldmask'][200*7:200*8,200*6:200*7].data
label_200 = np.load('/home/zhoutt/code/datasets/H8label2016_06_07_npy/20160730_0230_07_08.npz')['arr_0'][9,:,:]
dif_disk_200 = label_disk - label_200 
aa =  np.unique(dif_disk_200)
for i in aa:
    num = np.sum(dif_disk_200==i)
    print(i,num)

0.0 40000


### 1.4 对比200*200尺寸label 与test_result

In [7]:
result_200 = np.load('/home/zhoutt/code/datasets/results/200_disk/20160730_0230_08_08.npy')
label_200 = np.load('/home/zhoutt/code/datasets/H8label2016_06_07_npy/20160730_0230_08_08.npz')['arr_0'][9,:,:]
label_200[label_200==6] = 0
dif_disk_200 = result_200 - label_200
aa =  np.unique(dif_disk_200)
for i in aa:
    num = np.sum(dif_disk_200==i)
    print('dif_disk_200',num)

FileNotFoundError: [Errno 2] No such file or directory: '/home/zhoutt/code/datasets/results/200_disk/20160730_0230_08_08.npy'

### 1.5 对比200 result 与disk result

In [27]:
result_200 = np.load('/home/zhoutt/code/datasets/results/200_disk/20160730_0230_08_08.npy')
result_disk = np.load('/home/zhoutt/code/datasets/results/disk_splice_1/20160730_0230.npy')[200:400,200:400]
# label_200[label_200==6] = 0
dif_disk_200 = result_200 - result_disk
aa =  np.unique(dif_disk_200)
for i in aa:
    num = np.sum(dif_disk_200==i)
    print(i,num)

0 40000


## 2. 对比200数据集与256数据集，确保256数据集没有出错

### 2.1 测试200 与 256 6月7月数据

In [ ]:
data_200 = np.load('/home/zhoutt/code/datasets/H8label2016_06_07_npy/20160603_0230_01_01.npz')['arr_0'][8,:,:]
data_256 = np.load('/home/zhoutt/code/datasets/H8label2016_01_12_npy/06/20160603_0230_01_01.npz')['arr_0'][8,:,:][0:200,0:200]
# label_200[label_200==6] = 0
dif_disk_200 = data_200 - data_256
aa =  np.unique(dif_disk_200)
for i in aa:
    num = np.sum(dif_disk_200==i)
    print(i,num)

### 2.2 测试 npy 与 npz  dataset 是否完全一致

In [8]:
data_npy = np.load('/home/LVM_date/zhoutt/datasets/H8label2016_01_12_npy/12/20161203_0230_20_20.npy')[9,:,:]
data_npz = np.load('/home/LVM_date/zhoutt/datasets/H8label2016_01_12_npz/12/20161203_0230_20_20.npz')['arr_0'][9,:,:]
# label_200[label_200==6] = 0
dif_disk_200 = data_npy - data_npz
aa =  np.unique(dif_disk_200)
for i in aa:
    num = np.sum(dif_disk_200==i)
    print(i,num)
"/home/LVM_date/zhoutt/datasets/H8label2016_01_12_npy/03/20160303_1130_18_16.npy"==6


0.0 65536
